## 使用SageMaker的BYOS(Bring Your Oen Script)方式部署大语言模型

利用SageMaker部署大语言模型的原理如下：

<img src="imgs/sagemaker_deploy_model.jpg" style="width: 850px;"></img>

SageMaker提供了几种不同的Container来部署模型，[DJL](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-tutorials-deepspeed-djl.html)是AWS提供的用于部署大模型的容器。使用它，我们可以使用DeepSpeed、HuggingFace、FatserTransformer来进行加速。

<img src="imgs/djl.jpg" style="width: 850px;"></img>

目前不同的加速框架支持的模型如下（2023年3月）：

<img src="imgs/djl_models.jpg" style="width: 850px;"></img>

目前DeepSpeed已经支持LLaMA，并且性能提升达到3倍。

### 可部署的模型

使用DJL容器部署模型，对于一部分支持的模型，只需要通过定义DeepSpeedModel 或 HuggingFaceAccelerateModel 来部署即可，但是对于一些模型，可能需要设置 *trust_custom_code=True* ，或者需要自己的推理脚本，这时候，就使用自己的脚本，打包后传到S3，使用 DJL 的推理容器来进行部署。

这里，提供的部署脚本有：
1. chatglm2
2. cn-alpaca
3. baichuan2

如果需要其他的模型，可以使用类似的方式提供脚本。


### 下载模型到S3

如果是国内部署，从huggingface上下载模型可能会很慢，甚至经常因为超时而出错，所以可以先下载到本地后，上传到S3，然后从s3地址部署。

这里使用 *Baichuan2-13B-Chat-4bits* 模型，它是13B的模型，正常部署需要至少32GB显存，24GB显存的机器能部署，然后一推理就会OOM。但是这里使用4bit的量化版本，只需要16GB显存的机型，就能部署使用，而且模型的能力下降也不多，从官方提供的benchmark来看，只有微小的下降。

先将模型下载到本地。

In [76]:
# 需要先安装依赖
# %pip install huggingface_hub --upgrade --quiet

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import os

# - This will download the model into the ./model directory where ever the jupyter file is running
local_model_path = Path("/home/ec2-user/SageMaker/models/baichuan-inc--Baichuan2-13B-Chat-4bits")
local_model_path.mkdir(exist_ok=True)
model_name = "baichuan-inc/Baichuan2-13B-Chat-4bits"
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.bin", "*.txt", "*.model", "*.py"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=Path("/home/ec2-user/SageMaker/models"),
    local_dir_use_symlinks=False,
    local_dir=local_model_path,
    allow_patterns=allow_patterns,
)

In [ ]:
# upload to s3
key_prefix="mt_models_uploaded/baichuan-inc--Baichuan2-13B-Chat-4bits"

model_artifact = sess.upload_data(path=model_download_path, key_prefix=key_prefix)
print(f"Model uploaded to --- > {model_artifact}")
print(f"You can set option.s3url={model_artifact}")

将上传的地址，如“s3://sagemaker-us-east-1-568765279027/mt_models_uploaded/baichuan-inc--Baichuan2-13B-Chat-4bits” 提换到 djl-baichuan2-13b-4bits 目录中的 option.s3url 路径
```
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.s3url = s3://sagemaker-us-east-1-568765279027/mt_models_uploaded/baichuan-inc--Baichuan2-13B-Chat-4bits/
```

### 部署

准备：
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [1]:
# 如果需要，更新sagemaker和 aws python sdk boto3
# %pip install --upgrade awscli boto3
# %pip install --upgrade sagemaker

In [1]:
from sagemaker import __version__
print(__version__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
2.196.0


In [2]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
region = sagemaker_session._region_name # region name of the current environment
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
print(bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker-us-east-1-568765279027


接下来，我们使用Sagemaker进行模型部署。

In [3]:
# 我们部署 chatglm2 
# model_code = "chatglm2"
# model_code = "cn-alpaca2-7b"
model_code = "baichuan2"
model_dir = "baichuan2-13b-4bits"

In [4]:

# engine = "hf"  # using huggingface
engine = "py"  # using Python
engine = "ds"  # using deepspeed
# engine = "ft"  # using fastertransformer

dir_name = "djl-" + engine + "-" + model_code
dir_name = "djl-" + model_dir
dir_name

'djl-baichuan2-13b-4bits'

选择要使用的镜像，最新版是"0.23.0"

In [5]:
from sagemaker import image_uris
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed",
    region=region,
    version="0.24.0"
)
print(inference_image_uri)

763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.24.0-deepspeed0.10.0-cu118


In [6]:
# 需要将文件夹打包传到S3上，部署的时候需要

!rm {dir_name}.tar.gz
!cd LLM_bc2_4bit_stream_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf {dir_name}.tar.gz {dir_name}/ 

/bin/sh: line 0: cd: LLM_bc2_4bit_stream_deploy_code: No such file or directory
djl-baichuan2-13b-4bits/
djl-baichuan2-13b-4bits/.ipynb_checkpoints/
djl-baichuan2-13b-4bits/.ipynb_checkpoints/serving-checkpoint.properties
djl-baichuan2-13b-4bits/.ipynb_checkpoints/model-checkpoint.py
djl-baichuan2-13b-4bits/.ipynb_checkpoints/requirements-checkpoint.txt
djl-baichuan2-13b-4bits/serving.properties
djl-baichuan2-13b-4bits/requirements.txt
djl-baichuan2-13b-4bits/model.py


In [7]:
s3_code_prefix = (
    "deploy_code/" + dir_name  # folder within bucket where code artifact will go
)
s3_code_prefix

'deploy_code/djl-baichuan2-13b-4bits'

In [8]:
djl_s3_code_artifact = sagemaker_session.upload_data(dir_name + ".tar.gz", bucket, s3_code_prefix)

In [15]:
endpoint_name = 'mt-'+dir_name+'-g5'
endpoint_name

'mt-djl-baichuan2-13b-4bits-g5'

然后部署该模型为 Sagemaker endpoint

In [16]:
from sagemaker.model import Model

instance_type = "ml.g5.2xlarge"

model = Model(
    name=endpoint_name + "-model",
    image_uri=inference_image_uri, 
    model_data=djl_s3_code_artifact, 
    role=role
)

model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=600
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
------------------!

In [17]:
endpoint_name

'mt-djl-baichuan2-13b-4bits-g5'

In [18]:
from sagemaker import serializers, deserializers

predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(), 
    deserializer=deserializers.JSONDeserializer()
)

In [19]:
# from prompt_generator import generate_prompt as gen_prompt
import prompt_generator as pg
import importlib

importlib.reload(pg)

<module 'prompt_generator' from '/home/ec2-user/SageMaker/LLM_Workshop/prompt_generator.py'>

In [20]:
payload = pg.create_payload("2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案：", instruct='信息抽取', model_code=model_code)
payload

{'inputs': [{'role': 'system', 'content': '信息抽取'},
  {'role': 'user', 'content': '2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案：'}],
 'parameters': {'do_sample': False,
  'temperature': 0.3,
  'top_k': 50,
  'max_new_tokens': 2048}}

In [21]:
%%time
# payload = pg.create_payload("信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案：", model_code)
payload = pg.create_payload("2022年世界杯的冠军是阿根廷队伍，C罗是MVP\n问题：国家名，人名\n答案：", instruct='信息抽取', model_code=model_code)
predictor.predict(payload)

CPU times: user 24.5 ms, sys: 843 µs, total: 25.3 ms
Wall time: 4.21 s


{'outputs': '国家名：阿根廷\n人名：C罗（克里斯蒂亚诺·罗纳尔多）'}

如果部署过程中出现错误，部署失败，通过下面的方式删除endpoint以及相应的模型。

In [77]:
endpoint_name

'mt-djl-baichuan2-13b-4bits-g4dn'

In [17]:
from sagemaker import serializers, deserializers

del_predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sagemaker_session
)
# del_predictor.delete_model()
# del_predictor.delete_endpoint()


In [43]:
inputs= [
    pg.create_payload("写一首关于交通信号灯的诗", model_code=model_code),
    pg.create_payload("陨石为什么总能落在陨石坑里?", model_code=model_code),
    pg.create_payload("为什么爸妈结婚没叫我参加婚礼?", model_code=model_code)
]
response = predictor.predict(inputs[0])
print("\n\n问题: ", inputs[0]["inputs"], "\n回答:\n", response["outputs"])
response = predictor.predict(inputs[1])
print("\n\n问题: ", inputs[1]["inputs"], "\n回答:\n", response["outputs"])
response = predictor.predict(inputs[2])
print("\n\n问题: ", inputs[2]["inputs"], "\n回答:\n", response["outputs"])



问题:  [{'role': 'user', 'content': '写一首关于交通信号灯的诗'}] 
回答:
 在这繁华的城市里，
交通信号灯如舞者般翩翩起舞。
红、黄、绿三色交织，
为这喧嚣的世界描绘出一幅画卷。

红灯亮起，如同烈焰燃烧，
警示着行人们暂停脚步。
黄灯闪烁，犹如黎明前的曙光，
提醒着我们即将进入新的时光。

绿灯闪耀，希望之光洒满大地，
我们欢快地穿越马路，奔向远方。
在这瞬息万变的世界里，
交通信号灯是我们生活的指引。

它们无声地守护着每一个路口，
为我们划分出安全的道路。
无论白天还是黑夜，
它们始终屹立在那里，不离不弃。

当我们感叹城市的繁华与喧嚣，
不要忘了这些默默付出的信号灯。
它们用红、黄、绿三种色彩，
为我们描绘出一个和谐的交通世界。

让我们向这些无声的守护者致敬，
感谢它们为我们的生活带来安宁。
在这繁华的都市中，
交通信号灯是我们的守护天使。


问题:  [{'role': 'user', 'content': '陨石为什么总能落在陨石坑里?'}] 
回答:
 实际上，并不是所有的陨石都会落在陨石坑中。然而，大多数情况下，当陨石撞击地球表面时，它会形成一个坑洞，这个坑洞通常被称为陨石坑。这是因为陨石的重量和速度使其对地表产生了足够的破坏力，从而改变了周围的物质并形成了这个坑洞。

在某些情况下，陨石可能会在陆地或水面上降落，而不是在固体表面上。在这种情况下，陨石可能会与地表的物质混合在一起，或者被水流带走。因此，在这些情况下，我们可能不会看到明显的陨石坑。

总的来说，虽然并非所有陨石都会落在陨石坑中，但大多数陨石在撞击地表时会形成陨石坑。


问题:  [{'role': 'user', 'content': '为什么爸妈结婚没叫我参加婚礼?'}] 
回答:
 因为你在当时还没有出生。当你出生时，你的父母已经结婚了。


### 通过Sagemaker Endpoint调用
我们已经将模型部署到了Sagemaker Endpoint上，我们就可以通过这个Endpoint名称，来调用模型进行推理，这样即使你停止了这个notebook，也能使用已经部署的模型。

In [22]:
import json
import boto3

client = boto3.client('runtime.sagemaker')
# sagemaker_endpoint_name = "mt-chatglm2-6b-ft-g4dn"
sagemaker_endpoint_name = "mt-djl-baichuan2-13b-4bits-g4dn"

def query_endpoint(encoded_json):
    response = client.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["outputs"]
    return generated_text


In [65]:
model_code

'baichuan2'

In [34]:
%%time

payload = pg.create_payload("2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案：", instruct='信息抽取', model_code=model_code)
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

国家名：阿根廷
人名：梅西
CPU times: user 24.7 ms, sys: 0 ns, total: 24.7 ms
Wall time: 1.19 s


In [76]:
%%time
payload = {"inputs": "写一首有关蓝天白云的四言诗。"}

answer = query_endpoint(json.dumps(payload).encode('utf-8'))
print(answer)
print(len(answer))

蓝天白云真美丽，白云蓝天真清新。
16
CPU times: user 3.76 ms, sys: 0 ns, total: 3.76 ms
Wall time: 686 ms


In [64]:
%%time

propmt = """使用下面的已知内容，简洁、直接的回答最后的问题。不要使用已有的知识，不要编造答案，答案越简洁越好。如果你从上下文中无法知道答案，就直接返回'根据已知信息无法回答该问题.'。

已知内容:
Q:水光针多久打一次呢?效果能维持多久?
A:水光针一般建议一个月左右打一次,大概就是28天,因为刚好皮肤的周期是这么久。
如果是第一次打水光，吸收比较快，也一些专家建议第一次和第二次间隔半个月,二十天左右就可以打第二次,之后在打就间隔一个月左右。
水光的功效主要看你加了什么成分，正常来说的话，一般的营养成分大概是一到两个月。可以维持,如果里面加了肉毒素的话，效果可以维持3个月
左右,所以像水光如果你不加肉毒素，是每个月都可以操作注射的，越打皮肤会越好。

Q:水光针做完一次可以维持多久?
A:对于肤质干燥且疏于保养的人而言，其效果大约能维持3个月左右，保养越好，效果维持时间相应越长。具体来说，水光针注射效果与接受注射的次数有关，连续进行三次注射后，效果可以维持1—2年左右，但根据个人体质、肤质、生活习惯等不同，具体的维持时间也会有所差异。

Q：水光针怎样打效果更好?
A:水光针的话，我认为用机器打，特别是这个34G的16针头去操作的话，效果一定会更好，然后再用到好的产品，那么就事半功倍了，然后术后的话可以多敷修复的面膜。然后在就是持续的去打，那么可以，达到一个一加一大于二的效果。

Q：水光针多久打一次呢?效果能维持多久?
A：水光针一般建议一个月左右打一次,大概就是28天,因为刚好皮肤的周期是这么久。
如果是第一次打水光，吸收比较快，也一些专家建议第一次和第二次间隔半个月,二十天左右就可以打第二次,之后在打就间隔一个月左右。
水光的功效主要看你加了什么成分，正常来说的话，一般的营养成分大概是一到两个月。可以维持,如果里面加了肉毒素的话，效果可以维持3个月
左右,所以像水光如果你不加肉毒素，是每个月都可以操作注射的，越打皮肤会越好。

Q:脸部、肚子溶脂针多久见效?
A:一般打完溶脂针的话就是五到七天会有效果，但是效果的显著程度因人而异，有的人打一次溶脂针效果就比较明显，但是有的人需要多次注射才会有明显的效果。除此之外，打完溶脂针以后，自身对手术部位的护理也会影响到溶脂针的见效速度与效果的好坏。
在打完溶脂针以后，要尽量的保持注射部位的清洁,在一天之内不能沾水，恢复期间也要避免外界的污染，更不能化妆，不要在这个注射部位涂抹外用的一些药物,以免刺激到注射的部位，影响溶脂针的一个效果，见效的时间也会因此而延长。
想要溶脂针的效果更好的,见效更快的话，多喝水也是可以起到一些的作用，虽然说打完溶脂针以后不能沾水，但是我们可以多喝水，最好是每天饮用一升以上的水，不仅可以促进身体的新陈代谢，水循环，对注射部位的恢复也会起到辅助的作用。
溶脂针就是注射一次,就有一次的效果,一般的话是按照疗程来注射，效果是最好的,疗程一般是六到八周左右，每两到三周就可以进行二次注射。
正常是第一次注射就能看到效果，第二次注射的时候的效果更加明显，第三次注射的时候就起到了一个巩固的作用。
"""
payload = pg.create_payload("水光针多久打一次?", instruct=propmt, model_code=model_code)

# {"inputs": propmt, "parameters": {'temperature': 0.01}}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

水光针一般建议一个月左右打一次。
CPU times: user 27.5 ms, sys: 821 µs, total: 28.4 ms
Wall time: 2.15 s


In [81]:
%%time
payload = {"inputs": "4加2等于几？"}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

4加2等于6。
CPU times: user 4.27 ms, sys: 0 ns, total: 4.27 ms
Wall time: 464 ms


In [82]:
%%time
# ChatGLM支持通过history传递聊天历史
payload = {
    "inputs": "再乘以3呢？",
    "history": [("数学计算：\n3加8等于几？\n答案：", "3加8等于11。")]}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

3加8等于11，再乘以3还是11。
CPU times: user 3.96 ms, sys: 0 ns, total: 3.96 ms
Wall time: 962 ms


### 删除 EndPoint

In [2]:
from sagemaker import serializers, deserializers

predictor = sagemaker.Predictor(
    endpoint_name='mt-djl-baichuan2-13b-4bits-g4dn', 
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(), 
    deserializer=deserializers.JSONDeserializer()
)

predictor.delete_model()
predictor.delete_endpoint()